<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis using SQL queries with the RSQLite R package. You will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your SQlIte connection

Load the 'RSQLite' library, and use the 'dbConnect( )' function as you did in the previous lab to establish the connection to your SQLite database.\
You are now ready to start running SQL queries using the RSQLite library as you did in Course 3.


In [1]:
# provide your solution here
#install.packages("RSQLite")
library("RSQLite")

Warning message:
“The packages `ellipsis` (>= 0.3.2) and `vctrs` (>= 0.3.8) are required as of rlang 1.0.0.”

In [3]:
conn <- dbConnect(RSQLite::SQLite(), "FinalDB.sqlite")

In [6]:
bike_df <- read.csv('bike_sharing_systems.csv')
forecast_df <- read.csv('cities_weather_forecast.csv')
cities_df <- read.csv('raw_worldcities.csv')
seoul_bike <- read.csv('seoul_bike_sharing_converted_normalized.csv')

In [11]:
dbWriteTable(conn, "WORLD_CITIES", cities_df, overwrite = TRUE, header = TRUE)
dbWriteTable(conn, "BIKE_SHARING_SYSTEMS", bike_df, overwrite = TRUE, header = TRUE)
dbWriteTable(conn, "CITIES_WEATHER_FORECAST", forecast_df, overwrite = TRUE, header = TRUE)
dbWriteTable(conn, "SEOUL_BIKE_SHARING", seoul_bike, overwrite = TRUE, header = TRUE)

Download the following csv files:

*   [WORLD_CITIES](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/world_cities.csv)

*   [BIKE_SHARING_SYSTEMS](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/bike_sharing_systems.csv)

*   [CITIES_WEATHER_FORECAST](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/cities_weather_forecast.csv)

*   [SEOUL_BIKE_SHARING](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/seoul_bike_sharing.csv)

and load the csv's  into 4 tables as mentioned below

*   SEOUL_BIKE_SHARING

*   CITIES_WEATHER_FORECAST

*   BIKE_SHARING_SYSTEMS

*   WORLD_CITIES

> Hint : Use the read_csv() function and dbWriteTable() functions


## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [12]:
# provide your solution here
dbGetQuery(conn, "SELECT COUNT() FROM SEOUL_BIKE_SHARING")

COUNT()
<int>
8465


## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [14]:
# provide your solution here
dbGetQuery(conn,"SELECT COUNT(HOUR) FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT != 0")

COUNT(HOUR)
<int>
8462


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [16]:
# provide your solution here
dbGetQuery(conn,"SELECT * FROM CITIES_WEATHER_FORECAST WHERE FORECAST_DATETIME = '2022-08-26 03:00:00' AND city ='Seoul'")

CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,FORECAST_DATETIME,SEASON
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<chr>,<chr>
Seoul,Clouds,10000,27.67,27.67,27.67,1006,51,1.06,233,2022-08-26 03:00:00,Summer


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [17]:
# provide your solution here
dbGetQuery(conn,"SELECT DISTINCT SEASONS FROM SEOUL_BIKE_SHARING")

SEASONS
<chr>
Winter
Spring
Summer
Autumn


## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [34]:
# provide your solution here
dbGetQuery(conn,"SELECT DATE AS FIRST_DATE FROM SEOUL_BIKE_SHARING LIMIT 1")
dbGetQuery(conn,"SELECT DATE AS LAST_DATE FROM SEOUL_BIKE_SHARING LIMIT 1 OFFSET 8464")

FIRST_DATE
<chr>
01/12/2017


LAST_DATE
<chr>
30/11/2018


## Task 6 - Subquery - 'all-time high'

#### determine which date and hour had the most bike rentals.


### Solution 6


In [35]:
# provide your solution here
dbGetQuery(conn, "SELECT DATE, HOUR FROM SEOUL_BIKE_SHARING WHERE RENTED_BIKE_COUNT=(SELECT MAX(RENTED_BIKE_COUNT) FROM SEOUL_BIKE_SHARING)")

DATE,HOUR
<chr>,<int>
19/06/2018,18


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [42]:
seoul_bike_denorm <- read.csv("seoul_bike_sharing_converted.csv")

In [43]:
dbWriteTable(conn,"SEOUL_BIKE_SHARING_DENORM",seoul_bike_denorm, overwrite = TRUE, header = TRUE)

In [62]:
# provide your solution here
dbGetQuery(conn,"SELECT SEASONS, avg(TEMPERATURE) AS AVERAGE_TEMPERATURE, avg(RENTED_BIKE_COUNT) 
                 AS AVERAGE_BIKE_RENTALS FROM SEOUL_BIKE_SHARING_DENORM GROUP BY SEASONS")
dbGetQuery(conn,"SELECT * FROM SEOUL_BIKE_SHARING_DENORM ORDER BY RENTED_BIKE_COUNT DESC LIMIT 10")
      

SEASONS,AVERAGE_TEMPERATURE,AVERAGE_BIKE_RENTALS
<chr>,<dbl>,<dbl>
Autumn,13.821580,924.1105
Spring,13.021685,746.2542
Summer,26.587711,1034.0734
Winter,-2.540463,225.5412


DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,⋯,HOUR_6,HOUR_7,HOUR_8,HOUR_9,SEASONS_AUTUMN,SEASONS_SPRING,SEASONS_SUMMER,SEASONS_WINTER,HOLIDAY_HOLIDAY,HOLIDAY_NO_HOLIDAY
<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
19/06/2018,3556,18,24.1,57,2.9,1301,15.0,0.56,0,⋯,0,0,0,0,0,0,1,0,0,1
21/06/2018,3418,18,27.8,43,3.0,1933,14.0,1.35,0,⋯,0,0,0,0,0,0,1,0,0,1
12/06/2018,3404,18,24.9,53,3.6,2000,14.6,1.28,0,⋯,0,0,0,0,0,0,1,0,0,1
20/06/2018,3384,18,27.0,55,3.1,1246,17.1,1.26,0,⋯,0,0,0,0,0,0,1,0,0,1
04/06/2018,3380,18,24.4,48,1.9,1998,12.6,0.56,0,⋯,0,0,0,0,0,0,1,0,0,1
22/06/2018,3365,18,29.3,27,3.4,1977,8.3,1.24,0,⋯,0,0,0,0,0,0,1,0,0,1
08/06/2018,3309,18,26.2,54,2.2,1183,16.1,0.88,0,⋯,0,0,0,0,0,0,1,0,0,1
10/09/2018,3298,18,25.9,42,1.1,2000,11.9,0.48,0,⋯,0,0,0,0,1,0,0,0,0,1
17/09/2018,3277,18,25.3,56,2.8,1992,15.8,0.54,0,⋯,0,0,0,0,1,0,0,0,0,1


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.

> Hint : Use the SQRT(AVG(col\*col) - AVG(col)\*AVG(col) ) function where col refers to your column name for finding the standard deviation


### Solution 8


In [49]:
# provide your solution here
dbGetQuery(conn,"SELECT SEASONS, avg(RENTED_BIKE_COUNT) AS AVERAGE_BIKE_RENTALS, min(RENTED_BIKE_COUNT) AS MINIMUM_BIKE_RENTALS, max(RENTED_BIKE_COUNT)
                 AS MAXIMUM_BIKE_RENTALS, sqrt((avg(RENTED_BIKE_COUNT*RENTED_BIKE_COUNT))-(avg(RENTED_BIKE_COUNT)*avg(RENTED_BIKE_COUNT))) AS 
                 STANDARD_DEVIATION_BIKE_RENTALS
                 FROM SEOUL_BIKE_SHARING_DENORM
                 GROUP BY SEASONS")

SEASONS,AVERAGE_BIKE_RENTALS,MINIMUM_BIKE_RENTALS,MAXIMUM_BIKE_RENTALS,STANDARD_DEVIATION_BIKE_RENTALS
<chr>,<dbl>,<int>,<int>,<dbl>
Autumn,924.1105,2,3298,617.3885
Spring,746.2542,2,3251,618.5247
Summer,1034.0734,9,3556,690.0884
Winter,225.5412,3,937,150.3374


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [53]:
# provide your solution here
average_weather <-dbGetQuery(conn,"SELECT SEASONS, avg(TEMPERATURE), avg(HUMIDITY), 
                 avg(VISIBILITY),
                 avg(WIND_SPEED), avg(DEW_POINT_TEMPERATURE),
                 avg(SOLAR_RADIATION), avg(RAINFALL), avg(SNOWFALL),
                 avg(RENTED_BIKE_COUNT) AS AVERAGE_BIKE_RENTALS
                 FROM SEOUL_BIKE_SHARING_DENORM
                 GROUP BY SEASONS
                 ORDER BY AVERAGE_BIKE_RENTALS DESC")

average_weather$RANK <- rank(-average_weather$AVERAGE_BIKE_RENTALS, ties.method = "min")
average_weather

SEASONS,avg(TEMPERATURE),avg(HUMIDITY),avg(VISIBILITY),avg(WIND_SPEED),avg(DEW_POINT_TEMPERATURE),avg(SOLAR_RADIATION),avg(RAINFALL),avg(SNOWFALL),AVERAGE_BIKE_RENTALS,RANK
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Summer,26.587711,64.98143,1501.745,1.609420,18.750136,0.7612545,0.25348732,0.00000000,1034.0734,1
Autumn,13.821580,59.04491,1558.174,1.492101,5.150594,0.5227827,0.11765617,0.06350026,924.1105,2
Spring,13.021685,58.75833,1240.912,1.857778,4.091389,0.6803009,0.18694444,0.00000000,746.2542,3
Winter,-2.540463,49.74491,1445.987,1.922685,-12.416667,0.2981806,0.03282407,0.24750000,225.5412,4


## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [58]:
# provide your solution here
dbGetQuery(conn, "SELECT b.CITY, w.COUNTRY, w.LAT, w.LNG, w.POPULATION, b.BICYCLES FROM WORLD_CITIES w, BIKE_SHARING_SYSTEMS b 
                  WHERE w.CITY=b.CITY AND w.CITY='Seoul'")

CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
Seoul,"Korea, South",37.5833,127,21794000,37500


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [59]:
# provide your solution here
dbGetQuery(conn, "SELECT b.CITY, w.COUNTRY, w.LAT, w.LNG, w.POPULATION, b.BICYCLES FROM WORLD_CITIES w, BIKE_SHARING_SYSTEMS b
                  WHERE w.CITY=b.CITY AND b.BICYCLES BETWEEN 15000 AND 20000")

CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
Beijing,China,39.9050,116.3914,19433000,16000
Ningbo,China,29.8750,121.5492,7639000,15000
Shanghai,China,31.1667,121.4667,22120000,19165
Weifang,China,36.7167,119.1000,9373000,20000
Zhuzhou,China,27.8407,113.1469,3855609,20000


In [ ]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>
<h4> Lakshmi Holla </h4>

## Other Contributor(s)

<h4>  Malika Singla </h4>

## Change log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
| 2022-03-03        | 1.0     | Lakshmi Holla | Created lab using sqlite3 |

## <h3 align="center"> © IBM Corporation 2022. All rights reserved. <h3/>
